In [ ]:
!pip install scrapy
!pip install fonduer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.4/286.4 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.1/247.1 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.7/97.7 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.3/145.3 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.1/68.1 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.1/175.1 kB 20.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.

In [ ]:
import scrapy
class WebSpider ( scrapy . Spider ):
  name = 'web_spider'
  start_urls = [
    'https :// example .com /',
  ]

  def parse ( self , response ) :
    # Check this page has needed data
    if response . css ('div.real - estate') . get () is not None :
      yield {
        'address': response.css('div.real-estate>div.address::text').get(),
        'price': response.css('div.real-estate>div.price::text').get(),
        'acreage': response.css('div.real-estate>div.acreage::text').get()
      }

    next_pages = response.css('li.next a::attr("href")').all()
    for next_page in next_pages :
      yield response.follow(next_page, self.parse)

In [ ]:
PARALLEL = 4 # assuming a quad - core machine
ATTRIBUTE = "real_estate_fonduer"
conn_string = 'postgresql://localhost:5432/' + ATTRIBUTE

from fonduer import Meta , init_logging

# Configure logging for Fonduer
init_logging(log_dir="logs")

session = Meta.init(conn_string).Session()


In [ ]:
from fonduer.parser.preprocessors import HTMLDocPreprocessor
from fonduer.parser import Parser

docs_path = 'data/html/'

max_docs = 100
doc_preprocessor = HTMLDocPreprocessor(docs_path, max_docs=max_docs)

corpus_parser = Parser(session, structural=True, lingual=True, language ="vi", tabular=True,)
% time corpus_parser.apply(doc_preprocessor, parallelism=PARALLEL)


In [ ]:
Number = mention_subclass("Number")
Address = mention_subclass("Address")
Unit = mention_subclass("Unit")
Date = mention_subclass("Date")
NumberCandidate = candidate_subclass("NumberCandidate", [Number])
AddressCandidate = candidate_subclass("AddressCandidate", [Address])
PriceCandidate = candidate_subclass("PriceCandidate", [Number, Unit])
DateCandidate = candidate_subclass("DateCandidate", [Date])

In [ ]:
mention_number = MentionNgrams(n_max=1)
regx_match_number = RegexMatchSpan(rgx='[0 -9]', search=True, full_match=False)

mention_unit = MentionNgrams(n_min=1, n_max=3)
match_unit = DictionaryMatch(d = price_unit)
mention_date = MentionNgrams(n_min=5, n_max=5)
match_date = RegexMatchSpan(rgx ='^[0 -3]?[0 -9][\/ -][0 -3]?[0 -9][\/ -](?:[0 -9]{2})?[0 -9]{2}$')


In [ ]:
def check_is_title (m: TemporaryContext):
  words = m.get_attrib_tokens()
  text = '␣'.join(words)
  if text.istitle():
    false_list = [',', '|', '-']
    for char in false_list:
      if char in words:
        return False
    return True
  elif text.isdigit():
    return True
  else :
    return False

class MentionAddressNgrams (Ngrams) :
  def __init__ (
      self, n_min: int = 1, n_max: int = 5 ,
      split_tokens: Collection[str] = [] ,
      address_name: Collection[str] = []
    ) -> None:
      self.address_name = address_name
      Ngrams.__init__(self, n_min = n_min, n_max = n_max, split_tokens = split_tokens)

  def apply (self, doc: Document) -> Iterator[TemporarySpanMention]:
    for sentence in doc.sentences:
      text_sent = sentence.text.lower().replace('_', '␣')
      count_address = 0
      for address in self.address_name:
        if address in text_sent:
          count_address += 1
        if count_address > 0:
          for ts in Ngrams.apply(self, sentence):
            yield ts
mention_address = MentionAddressNgrams(n_min=1, n_max=3, address_name = list_address_name)
match_address = LambdaFunctionMatcher(func = check_is_title)

In [ ]:
ABSTAIN = -1
DUONG = 0
PHUONG = 1
QUAN = 2
TINH = 3
DU_AN = 4
FALSE_ADDRESS = 5
from gmlib.address_tagger.parse import AddressTagger
tagger = AddressTagger ()
tagger.load_model('accent')

def lf_address_extract_model(c):
  span_mention = c.address.context
  sent_text = span_mention.sentence.text
  char_start = span_mention.char_start
  char_end = span_mention.char_end
  result = tagger.parser(sent_text)
  for entity in result ['entities']:
    if entity ['start'] == char_start and entity ['end']==char_end:
      return map_entity[entity['entity']]
  return ABSTAIN

def lf_in_dic(c):
  text_span = c.des[1].lower().replace('_', '␣')
  if text_span in key_address_name['tinh']:
    return TINH
  if text_span in key_address_name['quan']:
    return QUAN
  if text_span in key_address_name['phuong']:
    return PHUONG
  if text_span in key_address_name['duong']:
    return DUONG
  if text_span in key_address_name['du_an']:
    return DU_AN
  return ABSTAIN

In [ ]:
ABSTAIN = -1
DIEN_TICH = 0
SO_TANG = 1
SO_PHONG_NGU = 4
SO_PHONG_KHACH = 5
SO_PHONG_VE_SINH = 7
MAT_TIEN = 8
CHIEU_DAI = 9
CHIEU_RONG = 10
DO_RONG_DUONG = 11
SO_DIEN_THOAI = 12
NUMBER_FAILE = 13
map_entity_label = {
  "ABSTAIN": ABSTAIN ,
  "surface_size": DIEN_TICH ,
  "number_of_floors": SO_TANG ,
  "number_of_rooms": SO_PHONG_NGU ,
  "number_of_toilets": SO_PHONG_VE_SINH ,
  "surface_width": MAT_TIEN ,
  "surface_length": CHIEU_DAI ,
  "street_width": DO_RONG_DUONG ,
  "contact_phone": SO_DIEN_THOAI ,
  "NUMBER_FALSE": NUMBER_FAILE
}

from gmlib.content_extract import IE
content_ie = IE ()

def lf_number_extract_model(c):
  span_mention = c.address.context
  sent_text = span_mention.sentence.text
  char_start = span_mention.char_start
  char_end = span_mention.char_end
  result = IE.extract(sent_text)
  for entity in result ['entities']:
    if entity ['start'] == char_start and entity ['end']==char_end :
      if entity ['entity'] in map_entity_label :
        return map_entity_label[entity['entity']]
  return ABSTAIN

In [ ]:
NGAY_DANG = 0
FALSE_DATE = 1
date_labels = {
  "submission_date": NGAY_DANG ,
  "FALSE_DATE": FALSE_DATE ,
}
def lf_is_ngay_dang(c):
  pre_words = [word.lower() for word in c.des[0]]
  pre_text = '␣'.join(pre_words)
  pre_text = pre_text.replace ('_', '␣')
  if 'đăng' in pre_text :
    return NGAY_DANG
  if 'cập␣nhật' in pre_text :
    return NGAY_DANG
  return FALSE_DATE

In [ ]:
ABSTAIN = -1
GIA = 0
FALSE_GIA = 1
price_labels = {
  "price": GIA ,
  "FALSE_GIA": FALSE_GIA ,
}

def lf_is_not_false_gia(c):
  (number, unit) = c
  pre_words = [word.lower() for word in c.des[0]]
  pre_text = '␣'.join(pre_words)
  lower_unit = unit.context.get_span().lower()
  if lower_unit in ['triệu', 'tr']:
    if 'lỗ' in pre_text :
      return FALSE_GIA
    if '%' in pre_text :
      return FALSE_GIA
    if 'chiết_khấu' in pre_text :
      return FALSE_GIA
    if 'giảm' in pre_text :
      return FALSE_GIA
    if 'quà' in pre_text :
      return FALSE_GIA
    if 'tặng' in pre_text :
      return FALSE_GIA
    if 'voucher' in pre_text :
      return FALSE_GIA
    if 'trả␣trước' in pre_text :
      return FALSE_GIA
  if lower_unit in ['triệu/tháng', 'triệu/th', 'tr/th']:
    if 'trả_góp' in pre_text :
      return FALSE_GIA
  return GIA

In [ ]:
def ls_address_gold(c):
  document = c.document
  gold_label_address = document.meta['gold_label']['address']
  span_text = c.address.context.get_span().lower()
  for key in gold_label_address:
    if gold_label_address[key] == span_text:
      return map_label [key]
  return ABSTAIN


In [ ]:
docs = session.query(Document).order_by(Document.name).all()
ld = len(docs)

train_docs = set()
dev_docs = set()
test_docs = set()
splits = (0.8, 0.9)
data = [(doc.name, doc) for doc in docs]
data.sort(key=lambda x: x[0])
for i, (doc_name, doc ) in enumerate(data):
  if i < splits[0] * ld:
    train_docs.add(doc)
  elif i < splits[1] * ld:
    dev_docs.add(doc)
  else :
    test_docs.add(doc)

In [ ]:
featurization:
  textual:
    window_feature:
      size: 3
      combinations: True
      isolated: True
    word_feature:
      window: 7
  tabular:
    unary_features:
      attrib:
        - words
      get_cell_ngrams:
        max: 2
      get_head_ngrams:
        max: 2
      get_row_ngrams:
        max: 2
      get_col_ngrams:
        max: 2
learning:
  LogisticRegression:
    hidden_dim: 100
    bias: False

In [ ]:
for i, docs in enumerate([train_docs, dev_docs, test_docs]):
  candidate_extractor.apply( docs, split=i, parallelism=PARALLEL)

train_cands = candidate_extractor.get_candidates(split = 0)
dev_cands = candidate_extractor.get_candidates(split = 1)
test_cands = candidate_extractor.get_candidates(split = 2)

from fonduer.features import Featurizer

featurizer = Featurizer(session, [NumberCandidate, AddressCandidate, PriceCandidate, DateCandidate])

% time featurizer.apply(split=0, train=True, parallelism=PARALLEL)
% time F_train = featurizer.get_feature_matrices(train_cands)

% time featurizer.apply(split=1, parallelism=PARALLEL)
% time F_dev=featurizer.get_feature_matrices(dev_cands)

% time featurizer.apply(split=2, parallelism = PARALLEL)
% time F_test=featurizer.get_feature_matrices(test_cands)